In [4]:
%load_ext autoreload
%autoreload 2

import networkx
import numpy as np
import os.path
from random import choice
import tempfile
import typing as tp
import flybrainlab as fbl

import linecache
import os
import tracemalloc

import netpyne
from netpyne.specs import netParams, SimConfig
from netpyne import conversion, sim

from package.nlpToNetpyne import *

In [6]:
def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [36]:
my_client = fbl.get_client()
#res = my_client.executeNLPquery('show column C')
#res = my_client.executeNLPquery('show column A and column C')
#res = my_client.executeNLPquery('show column A and column B and column C')
#res = my_client.executeNLPquery('show column A and column B and column C and column D')
#res = my_client.executeNLPquery('show column A and column B and column C and column D and column E')
res = my_client.executeNLPquery('show column A and column B and column C and column D and column E and column F')

[FBL NLP 2023-06-16 08:31:57,301] NLP successfully parsed query.


In [37]:
networkParameters, simConfig = model_gen(client=my_client,
                                         res=res,
                                         filename="ColumnC",
                                         stim_targets={'Mi4_C': {'source': 'bkg', 'weight': 0.5, 'delay': 1, 'mech': 'exc'}},
                                         record_names=['L2_C', 'C2_C', 'Mi4_C'],
                                         maintain_morphology=False)

In [38]:
tracemalloc.start()

simulate(networkParameters, simConfig)

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

tracemalloc.stop()


Start time:  2023-06-16 08:33:54.527133

Creating network of 213 cell populations on 1 hosts...
  Number of cells on node 0: 213 
  Done; cell creation time = 0.46 s.
Making connections...
  Number of connections on node 0: 2557 
  Done; cell connection time = 1.44 s.
Adding stims...
  Number of stims on node 0: 1 
  Done; cell stims creation time = 0.00 s.
Recording 3 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 3.64 s; real-time ratio: 0.27.

Gathering data...
  Done; gather time = 0.22 s.

Analyzing...
  Cells: 213
  Connections: 2558 (12.01 per cell)
  Spikes: 2188 (10.27 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 3.64 s
Preparing spike data...
Plotting raster...
Plotting recorded cell traces ... cell
Plotting 2D representation of network cell locations and connections...
Plotting connectivity matrix...
  Done; plotting time = 20.91 s

Total time = 26.77 s
Top 10 lines
#1: C:\Users\NJPar\anaconda3\envs\flybrainlab\lib\sit

In [40]:
export_model(networkParameters, simConfig, 'columnC', "python")
export_model(networkParameters, simConfig, 'columnC', "neuroml")

script saved on columnC
Exporting the network to NeuroML 2, reference: columnC, connections: True, stimulations: True, format: xml
Is Mi15_F in odict_keys(['Mi15_F', 'Mi9_F', 'TmY5a_F', 'R7_F', 'Tm20_F', 'Mi1_F', 'L1_F', 'T1_F', 'L2_F', 'C2_F', 'R8_F', 'Tm25Y1_F', 'Mi4_F', 'L3_F', 'C3_F', 'L5_F', 'Dm2_F', 'TmY17_F', 'L4_F', 'T2_F', 'T4c_du_F', 'L1_E', 'T2a_F', 'T4a_fb_F', 'T3_F', 'CT1_F', 'T4b_bf_F', 'C3_E', 'Tm9_F', 'Tm5Y_F', 'Mi13_F', 'Tm1_F', 'Dm8_F', 'R8_E', 'C2_E', 'Tm25Y1_E', 'Tm4_F_ant', 'T4d_ud_F', 'Tm614_F', 'Tm2_F', 'Tm4_E_ant', 'Tm3_E_post', 'TmY5a_E', 'Tm2_E', 'L2_E', 'L5_E', 'Tm20_E', 'Tm3_E_ant', 'Mi4_E', 'R7_E', 'T1_E', 'Mi9_E', 'Mi1_E', 'Tm1_E', 'L3_E', 'T2_E', 'Mi13_E', 'T2a_E', 'Tm614_E', 'L4_E', 'Tm5Y_E', 'CT1_E', 'L5_D', 'T4a_fb_E', 'L3_D', 'Tm5a_E', 'L2_D', 'Dm8_E', 'Tm9_E', 'C2_D', 'L1_D', 'T4c_du_E', 'Dm2_E', 'TmY17_E', 'R8_D', 'T4d_ud_E', 'R7_D', 'Mi15_D', 'T4b_bf_E', 'T3_E', 'T2a_D', 'L4_D', 'Mi1_D', 'TmY3_D', 'C3_D', 'Tm3_D_post', 'Mi9_D', 'TmY17_D', 'Tm9_D', 

Exception: Noise = 0.5 in a spike generator is not yet supported for NeuroML export!